# **<span style="color:orange;">Import Libraries</span>**

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from feature_engine.encoding import RareLabelEncoder
from feature_engine.outliers import Winsorizer
from sklearn.model_selection import train_test_split
import math
import sklearn
import warnings

In [ ]:
pd.set_option('display.max_columns', None)
sklearn.set_config(transform_output='pandas')
warnings.filterwarnings("ignore")

# **<span style="color:orange;">Reading Data</span>**

In [100]:
def get_data():
    BASE_DIR = Path.cwd().parents[0]
    file_name = 'claims-severity-anonymized-raw.csv'
    return pd.read_csv(BASE_DIR / 'data' / 'allstate-claims-severity' / file_name, on_bad_lines="skip")

In [101]:
# Reading data from source
df = get_data()

In [102]:
# Applying Data Cleaning Steps
df = df.dropna(axis=0, ignore_index=True).drop('id', axis=1)

In [103]:
# Preview of data
df.head(5)

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,cat40,cat41,cat42,cat43,cat44,cat45,cat46,cat47,cat48,cat49,cat50,cat51,cat52,cat53,cat54,cat55,cat56,cat57,cat58,cat59,cat60,cat61,cat62,cat63,cat64,cat65,cat66,cat67,cat68,cat69,cat70,cat71,cat72,cat73,cat74,cat75,cat76,cat77,cat78,cat79,cat80,cat81,cat82,cat83,cat84,cat85,cat86,cat87,cat88,cat89,cat90,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,A,B,A,B,A,A,A,A,B,A,B,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,D,B,B,D,D,B,D,C,B,D,B,A,A,A,A,A,D,B,C,E,A,C,T,B,G,A,A,I,E,G,J,G,BU,BC,C,AS,S,A,O,LB,0.726300,0.245921,0.187583,0.789639,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,A,B,A,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,D,B,B,D,D,A,B,C,B,D,B,A,A,A,A,A,D,D,C,E,E,D,T,L,F,A,A,E,E,I,K,K,BI,CQ,A,AV,BM,A,O,DP,0.330514,0.737068,0.592681,0.614134,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,A,B,A,A,B,A,A,A,B,B,B,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,D,B,B,B,D,B,D,C,B,B,B,A,A,A,A,A,D,D,C,E,E,A,D,L,O,A,B,E,F,H,F,A,AB,DK,A,C,AF,A,I,GK,0.261841,0.358319,0.484196,0.236924,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,B,B,A,B,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,D,B,B,D,D,D,B,C,B,D,B,A,A,A,A,A,D,D,C,E,E,D,T,I,D,A,A,E,E,I,K,K,BI,CS,C,N,AE,A,O,DJ,0.321594,0.555782,0.527991,0.373816,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,A,B,A,B,A,A,A,A,B,B,A,B,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,D,B,D,B,D,B,B,C,B,B,C,A,A,A,B,H,D,B,D,E,E,A,P,F,J,A,A,D,E,K,G,B,H,C,C,Y,BM,A,K,CK,0.273204,0.159990,0.527991,0.473202,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


In [104]:
# List of Categorical Features
cat_cols = df.select_dtypes(include='O').columns.to_list()

# List of Numerical Features
num_cols = df.select_dtypes(include=float).columns.to_list()
num_cols.remove('loss')

print(cat_cols)
print("="*100)
print(num_cols)

['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18', 'cat19', 'cat20', 'cat21', 'cat22', 'cat23', 'cat24', 'cat25', 'cat26', 'cat27', 'cat28', 'cat29', 'cat30', 'cat31', 'cat32', 'cat33', 'cat34', 'cat35', 'cat36', 'cat37', 'cat38', 'cat39', 'cat40', 'cat41', 'cat42', 'cat43', 'cat44', 'cat45', 'cat46', 'cat47', 'cat48', 'cat49', 'cat50', 'cat51', 'cat52', 'cat53', 'cat54', 'cat55', 'cat56', 'cat57', 'cat58', 'cat59', 'cat60', 'cat61', 'cat62', 'cat63', 'cat64', 'cat65', 'cat66', 'cat67', 'cat68', 'cat69', 'cat70', 'cat71', 'cat72', 'cat73', 'cat74', 'cat75', 'cat76', 'cat77', 'cat78', 'cat79', 'cat80', 'cat81', 'cat82', 'cat83', 'cat84', 'cat85', 'cat86', 'cat87', 'cat88', 'cat89', 'cat90', 'cat91', 'cat92', 'cat93', 'cat94', 'cat95', 'cat96', 'cat97', 'cat98', 'cat99', 'cat100', 'cat101', 'cat102', 'cat103', 'cat104', 'cat105', 'cat106', 'cat107', 'cat108', 'cat109', 'cat110', 'cat111

In [ ]:
# Feature Name witg High Cardinality
high_cardinal_cols = [col for col in cat_cols if df[col].nunique()>10]

RareLabelEncoder(n_categories=2, max_n_categories=10, replace_with="Other").fit_transform(df[high_cardinal_cols]).head(5)

,cat99,cat100,cat101,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116
0,T,Other,G,A,I,E,G,J,G,Other,Other,C,AS,Other,A,O,Other
1,T,L,F,A,E,E,I,K,K,BI,Other,A,Other,BM,A,O,Other
2,Other,L,Other,B,E,F,H,F,Other,AB,Other,A,Other,Other,A,Other,Other
3,T,I,D,A,E,E,I,K,K,BI,CS,C,Other,AE,A,O,DJ
4,P,F,Other,A,D,E,Other,G,B,Other,Other,C,Other,BM,A,K,CK


📌 High-Cardinality Categorical Features

Several categorical features in the dataset exhibit high cardinality, with more than 10 unique labels. Such features contain a large number of distinct categories, which increases data sparsity and complexity in downstream modeling.
To address this, a Rare Label Encoding strategy was applied to consolidate infrequent categories into a single representative group. This approach reduces the effective cardinality of categorical variables while preserving the dominant patterns present in the data.

Overall, this transformation results in more compact and more interpretable categorical features, with reduced noise arising from low-frequency labels.

### **<span style="color:orange;">Creating Pipelines</span>**

In [93]:
categorical_pipe = Pipeline(
    steps=[
        ("rare_label", RareLabelEncoder(max_n_categories=10, replace_with='Other', missing_values='raise', n_categories=1)),
        ("ohe", OneHotEncoder(sparse_output=False))
    ]
)
categorical_pipe

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('rare_label', ...), ('ohe', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,tol,0.05
,n_categories,1
,max_n_categories,10
,replace_with,'Other'
,variables,None
,missing_values,'raise'
,ignore_format,False


In [94]:
numerical_pipe = Pipeline(
    steps=[
        ("scaling", StandardScaler())
    ]
)
numerical_pipe

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('scaling', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"copy copy: bool, default=TrueIf False, try to avoid a copy and do inplace scaling instead.This is not guaranteed to always work inplace; e.g. if the data isnot a NumPy array or scipy.sparse CSR matrix, a copy may still bereturned.",True
,"with_mean with_mean: bool, default=TrueIf True, center the data before scaling.This does not work (and will raise an exception) when attempted onsparse matrices, because centering them entails building a densematrix which in common use cases is likely to be too large to fit inmemory.",True
,"with_std with_std: bool, default=TrueIf True, scale the data to unit variance (or equivalently,unit standard deviation).",True


In [95]:
preprocessing_pipe = ColumnTransformer(
    transformers=[
        ('cat_pipe', categorical_pipe, cat_cols),
        ('num_pipe', numerical_pipe, num_cols)
    ]
)

In [96]:
preprocessing_pipe

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('cat_pipe', ...), ('num_pipe', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_name

### **<span style="color:orange;">Train-Test Split</span>**

In [97]:
X = df.drop('loss', axis=1)
y = df['loss']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [98]:
preprocessing_pipe.fit_transform(X_train, y_train)

,cat_pipe__cat1_A,cat_pipe__cat1_B,cat_pipe__cat2_A,cat_pipe__cat2_B,cat_pipe__cat3_A,cat_pipe__cat3_B,cat_pipe__cat4_A,cat_pipe__cat4_B,cat_pipe__cat5_A,cat_pipe__cat5_B,cat_pipe__cat6_A,cat_pipe__cat6_B,cat_pipe__cat7_A,cat_pipe__cat7_Other,cat_pipe__cat8_A,cat_pipe__cat8_B,cat_pipe__cat9_A,cat_pipe__cat9_B,cat_pipe__cat10_A,cat_pipe__cat10_B,cat_pipe__cat11_A,cat_pipe__cat11_B,cat_pipe__cat12_A,cat_pipe__cat12_B,cat_pipe__cat13_A,cat_pipe__cat13_B,cat_pipe__cat14_A,cat_pipe__cat14_Other,cat_pipe__cat15_A,cat_pipe__cat15_Other,cat_pipe__cat16_A,cat_pipe__cat16_Other,cat_pipe__cat17_A,cat_pipe__cat17_Other,cat_pipe__cat18_A,cat_pipe__cat18_Other,cat_pipe__cat19_A,cat_pipe__cat19_Other,cat_pipe__cat20_A,cat_pipe__cat20_Other,cat_pipe__cat21_A,cat_pipe__cat21_Other,cat_pipe__cat22_A,cat_pipe__cat22_Other,cat_pipe__cat23_A,cat_pipe__cat23_B,cat_pipe__cat24_A,cat_pipe__cat24_Other,cat_pipe__cat25_A,cat_pipe__cat25_B,cat_pipe__cat26_A,cat_pipe__cat26_B,cat_pipe__cat27_A,cat_pipe__cat27_B,cat_pipe__cat28_A,cat_pipe__cat28_Other,cat_pipe__cat29_A,cat_pipe__cat29_Other,cat_pipe__cat30_A,cat_pipe__cat30_Other,cat_pipe__cat31_A,cat_pipe__cat31_Other,cat_pipe__cat32_A,cat_pipe__cat32_Other,cat_pipe__cat33_A,cat_pipe__cat33_Other,cat_pipe__cat34_A,cat_pipe__cat34_Other,cat_pipe__cat35_A,cat_pipe__cat35_Other,cat_pipe__cat36_A,cat_pipe__cat36_B,cat_pipe__cat37_A,cat_pipe__cat37_B,cat_pipe__cat38_A,cat_pipe__cat38_B,cat_pipe__cat39_A,cat_pipe__cat39_Other,cat_pipe__cat40_A,cat_pipe__cat40_Other,cat_pipe__cat41_A,cat_pipe__cat41_Other,cat_pipe__cat42_A,cat_pipe__cat42_Other,cat_pipe__cat43_A,cat_pipe__cat43_Other,cat_pipe__cat44_A,cat_pipe__cat44_B,cat_pipe__cat45_A,cat_pipe__cat45_Other,cat_pipe__cat46_A,cat_pipe__cat46_Other,cat_pipe__cat47_A,cat_pipe__cat47_Other,cat_pipe__cat48_A,cat_pipe__cat48_Other,cat_pipe__cat49_A,cat_pipe__cat49_Other,cat_pipe__cat50_A,cat_pipe__cat50_B,cat_pipe__cat51_A,cat_pipe__cat51_Other,cat_pipe__cat52_A,cat_pipe__cat52_Other,cat_pipe__cat53_A,cat_pipe__cat53_B,cat_pipe__cat54_A,cat_pipe__cat54_Other,cat_pipe__cat55_A,cat_pipe__cat55_Other,cat_pipe__cat56_A,cat_pipe__cat56_Other,cat_pipe__cat57_A,cat_pipe__cat57_Other,cat_pipe__cat58_A,cat_pipe__cat58_Other,cat_pipe__cat59_A,cat_pipe__cat59_Other,cat_pipe__cat60_A,cat_pipe__cat60_Other,cat_pipe__cat61_A,cat_pipe__cat61_Other,cat_pipe__cat62_A,cat_pipe__cat62_Other,cat_pipe__cat63_A,cat_pipe__cat63_Other,cat_pipe__cat64_A,cat_pipe__cat64_Other,cat_pipe__cat65_A,cat_pipe__cat65_Other,cat_pipe__cat66_A,cat_pipe__cat66_Other,cat_pipe__cat67_A,cat_pipe__cat67_Other,cat_pipe__cat68_A,cat_pipe__cat68_Other,cat_pipe__cat69_A,cat_pipe__cat69_Other,cat_pipe__cat70_A,cat_pipe__cat70_Other,cat_pipe__cat71_A,cat_pipe__cat71_B,cat_pipe__cat72_A,cat_pipe__cat72_B,cat_pipe__cat73_A,cat_pipe__cat73_B,cat_pipe__cat73_Other,cat_pipe__cat74_A,cat_pipe__cat74_Other,cat_pipe__cat75_A,cat_pipe__cat75_B,cat_pipe__cat75_Other,cat_pipe__cat76_A,cat_pipe__cat76_Other,cat_pipe__cat77_D,cat_pipe__cat77_Other,cat_pipe__cat78_B,cat_pipe__cat78_Other,cat_pipe__cat79_B,cat_pipe__cat79_D,cat_pipe__cat79_Other,cat_pipe__cat80_B,cat_pipe__cat80_D,cat_pipe__cat80_Other,cat_pipe__cat81_B,cat_pipe__cat81_D,cat_pipe__cat81_Other,cat_pipe__cat82_A,cat_pipe__cat82_B,cat_pipe__cat82_D,cat_pipe__cat82_Other,cat_pipe__cat83_A,cat_pipe__cat83_B,cat_pipe__cat83_D,cat_pipe__cat83_Other,cat_pipe__cat84_A,cat_pipe__cat84_C,cat_pipe__cat84_Other,cat_pipe__cat85_B,cat_pipe__cat85_Other,cat_pipe__cat86_B,cat_pipe__cat86_C,cat_pipe__cat86_D,cat_pipe__cat86_Other,cat_pipe__cat87_B,cat_pipe__cat87_D,cat_pipe__cat87_Other,cat_pipe__cat88_A,cat_pipe__cat88_D,cat_pipe__cat88_Other,cat_pipe__cat89_A,cat_pipe__cat89_Other,cat_pipe__cat90_A,cat_pipe__cat90_B,cat_pipe__cat90_Other,cat_pipe__cat91_A,cat_pipe__cat91_B,cat_pipe__cat91_G,cat_pipe__cat91_Other,cat_pipe__cat92_A,cat_pipe__cat92_H,cat_pipe__cat92_Other,cat_pipe__cat93_C,cat_pipe__cat93_D,cat_pipe__cat93_Other,cat_pipe__cat94_B,cat_pipe__cat94_C,cat_pipe__cat94